In [1]:
import torch

from skimage.io import imread
from skimage import img_as_ubyte
from sigver.preprocessing.normalize import (
    normalize_image, resize_image,
    crop_center, preprocess_signature)

# Functions to load the CNN model
from sigver.featurelearning.models import SigNet

# Functions for plotting:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['image.cmap'] = 'Greys'

In [2]:
def load_signature(path):
    return img_as_ubyte(imread(path, as_gray=True))

def norm_resize_crop(original):
    normalized = 255 - normalize_image(original, (952, 1360))
    resized = resize_image(normalized, (170, 242))
    cropped = crop_center(resized, (150,220))
    return cropped

# Inputs need to have 4 dimensions (batch x channels x height x width), and also be between [0, 1]
def encoding(img):
    canvas_size = (952, 1360)
    img = norm_resize_crop(img)
    img = torch.tensor(preprocess_signature(img, canvas_size) )

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    state_dict, _, _ = torch.load('models/signet.pth')
    base_model = SigNet().to(device).eval()
    base_model.load_state_dict(state_dict)

    img = img.view(-1, 1, 150, 220).float().div(255)
    with torch.no_grad():
        img_features = base_model(img.to(device))
    return img_features

In [10]:
imgpath='data/b1.png'
original=load_signature(imgpath)
feature1  = encoding(original)
print(feature1)

imgpath='data/b2.png'
original=load_signature(imgpath)
feature2  = encoding(original)
print(feature2)

print(torch.norm(feature1-feature2))

tensor([[0.0000, 0.0000, 0.1949,  ..., 0.3808, 0.5881, 0.0000]])
tensor([[0.0000, 0.0000, 0.1593,  ..., 0.3981, 0.5870, 0.0000]])
tensor(0.5888)
